In [1]:
import os, sys, math, io
import numpy as np
import pandas as pd
import multiprocessing as mp
import bson
import struct
from PIL import Image
import time
import shutil

%matplotlib inline
import matplotlib.pyplot as plt

from collections import defaultdict
from tqdm import *


First load the lookup tables from the CSV files (you don't need to do this if you just did all the steps from part 1).

In [2]:
train_offsets_df = pd.read_csv("train_offsets.csv", index_col=0)
train_images_df = pd.read_csv("train_images_all.csv", index_col=0)
val_images_df = pd.read_csv("val_images_all.csv", index_col=0)

/home/bowen/anaconda3/envs/cdiscount/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
import torch
import torch.nn as nn
from torch.nn import init
from torch.autograd import Variable
import torchvision
import torchvision.transforms as T
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

In [4]:
class Dataset(object):
    def __init__(self, bson_file, images_df, offsets_df, transformer, train=True):
        self.file = bson_file
        self.images_df = images_df
        self.offsets_df = offsets_df
        self.transformer = transformer
        self.train = train

    def __getitem__(self, data):
        bson_img, y = data
        image = io.BytesIO(bson_img)
        img = Image.open(image)
        x = self.transformer(img)
        if self.train:
            return x, y
        else:
            return x

    def __len__(self):
        return len(self.images_df)

In [5]:
class MyBatchSampler(object):
    """Wraps another sampler to yield a mini-batch of indices.

    Args:
        sampler (Sampler): Base sampler.
        batch_size (int): Size of mini-batch.
        drop_last (bool): If ``True``, the sampler will drop the last batch if
            its size would be less than ``batch_size``

    Example:
        >>> list(BatchSampler(range(10), batch_size=3, drop_last=False))
        [[0, 1, 2], [3, 4, 5], [6, 7, 8], [9]]
        >>> list(BatchSampler(range(10), batch_size=3, drop_last=True))
        [[0, 1, 2], [3, 4, 5], [6, 7, 8]]
    """

    def __init__(self, sampler, batch_size, drop_last):
        self.sampler = sampler
        self.batch_size = batch_size
        self.drop_last = drop_last
        self.data_source = sampler.data_source

    def __getitem__(self, idx):
        image_row = self.data_source.images_df.iloc[idx]
        product_id = image_row["product_id"]
        offset_row = self.data_source.offsets_df.loc[product_id]
        # Random access this product's data from the BSON file.
        self.data_source.file.seek(offset_row["offset"])
        item_data = self.data_source.file.read(offset_row["length"])
        item = bson.BSON.decode(item_data)
        img_idx = image_row["img_idx"]
        
        return item["imgs"][img_idx]["picture"], image_row["category_idx"]
    
    def __iter__(self):
        batch_data = []
        for idx in self.sampler:
            data = self[idx]
            batch_data.append(data)
    
            if len(batch_data) == self.batch_size:
                yield batch_data
                batch_data = []
        if len(batch_data) > 0 and not self.drop_last:
            yield batch_data

    def __len__(self):
        if self.drop_last:
            return len(self.sampler) // self.batch_size
        else:
            return (len(self.sampler) + self.batch_size - 1) // self.batch_size


In [6]:
data_dir = "./input/"
train_bson_path = os.path.join(data_dir, "train.bson")
train_bson_file = open(train_bson_path, "rb")

In [8]:
mean, std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
transformer_train = T.Compose([T.RandomHorizontalFlip(), 
                             T.ToTensor(),T.Normalize(mean=mean, std=std)])
transformer_val = T.Compose([T.ToTensor(),T.Normalize(mean=mean, std=std)])

Create a generator for training and a generator for validation.

In [9]:
dataset_train = Dataset(train_bson_file, train_images_df, train_offsets_df, transformer_train, train=True)
dataset_val = Dataset(train_bson_file, val_images_df, train_offsets_df, transformer_val, train=True)

In [10]:
print(len(dataset_train), len(dataset_val))

12129141 242152


In [11]:
batch_size = 256
batch_sampler_train = MyBatchSampler(batch_size=batch_size, sampler=sampler.RandomSampler(dataset_train), 
                                     drop_last=False)
batch_sampler_val = MyBatchSampler(batch_size=batch_size, sampler=sampler.RandomSampler(dataset_val), 
                                     drop_last=False)

In [23]:
loader_train = DataLoader(dataset=dataset_train, batch_sampler=batch_sampler_train, num_workers=4, pin_memory=True)
loader_val = DataLoader(dataset=dataset_val, batch_sampler=batch_sampler_val, num_workers=4, pin_memory=True)

In [24]:
print(len(loader_train), len(loader_val))

47380 946


## How fast is the generator? Create a single batch:

In [25]:
itr = iter(loader_train)

In [26]:
%time bx, by = next(itr)
print(bx.size(), by.size())

CPU times: user 84 ms, sys: 8 ms, total: 92 ms
Wall time: 132 ms
torch.Size([256, 3, 180, 180]) torch.Size([256])


# Part 3: Training

**ResNet50**

In [ ]:
model = torchvision.models.resnet50(pretrained=True)
model.avgpool = nn.AvgPool2d(kernel_size = 6)
model.fc = nn.Linear(in_features=2048, out_features=5270) #or 49 + 483 + 5270
for layer in [model.conv1, model.bn1, model.relu, model.maxpool, model.layer1, model.layer2, model.layer3]:
    for param in layer.parameters():
        param.requires_grad = False

In [ ]:
init.kaiming_normal(model.fc.weight.data)
model.fc.bias.data.zero_()
model.cuda()


In [ ]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def adjust_learning_rate(lr, optimizer, epoch, denominator = 2):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = lr * (0.1 ** (epoch // denominator))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    batch_size = target.size(0)
    _, pred = output.max(dim=1)
    correct = pred.eq(target)
    res = []
    for k in topk:
        correct_k = correct.float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

In [ ]:
def train(train_loader, model, criterion, optimizer, epoch, print_freq = 500):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    loss_log = []
    acc_log = []
    
    # switch to train mode
    model.train()

    end = time.time()
    for i, (img, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        target = target.cuda(async=True)
        img = img.cuda(async=True)
        img_var = Variable(img)
        target_var = Variable(target)

        # compute output
        output = model(img_var)
        loss = criterion(output, target_var)
        # measure accuracy and record loss
        prec1 = accuracy(output.data, target, topk=(1, ))[0]#only need top1
        losses.update(loss.data[0], img.size(0)) #[0] to take out the float inside torch.Tensor
        top1.update(prec1[0], img.size(0))
        loss_log.append(losses.val)
        acc_log.append(top1.val)

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                   epoch, i, len(train_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))
#         if i == 10000:
#             break
    return loss_log, acc_log

In [ ]:
def validate(val_loader, model, print_freq=50):
    batch_time = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (img, target) in enumerate(val_loader):
        
        target = target.cuda(async=True)
        img = img.cuda(async=True)
        img_var = Variable(img, volatile=True)
        target_var = Variable(target, volatile=True)

        # compute output
        output = model(img_var)

        # measure accuracy and record loss
        prec1 = accuracy(output.data, target, topk=(1,))[0]#only need top1
        top1.update(prec1[0], img.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % print_freq == 0:
            print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                   i, len(val_loader), batch_time=batch_time, top1=top1))

    print(' * Prec@1 {top1.avg:.3f}'.format(top1=top1))

    return top1.avg

In [ ]:
if __name__ == '__main__':
    best_prec1 = 64
    criterion = nn.CrossEntropyLoss().cuda()
    lr = 1e-2
    optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr = lr, momentum=0.9, 
                          weight_decay=0)
    resume = None
    start_epoch = 0
    epochs = 2
    arch = 'resnet50'

    if resume:
        if os.path.isfile(resume):
            print("=> loading checkpoint '{}'".format(resume))
            checkpoint = torch.load(resume)
            start_epoch = checkpoint['epoch']
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(resume, checkpoint['epoch']))
        else:
            print("=> no checkpoint found at '{}'".format(resume))

    for epoch in range(start_epoch, epochs):
        
        adjust_learning_rate(lr=lr, optimizer=optimizer, epoch=epoch, denominator=1)

        # train for one epoch
        loss_log, acc_log = train(train_loader=loader_train, model=model, criterion=criterion,
                                  optimizer=optimizer, epoch=epoch)

        # evaluate on validation set
        prec1 = validate(val_loader=loader_val, model=model)

        # remember best prec@1 and save checkpoint
        is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)
        save_checkpoint({
            'epoch': epoch + 1,
            'arch': arch,
            'state_dict': model.state_dict(),
            'best_prec1': best_prec1,
            'optimizer' : optimizer.state_dict(),
        }, is_best)

        #plot loss and acc
        fig = plt.figure(figsize = (6,3), dpi = 1200)
        loss_log = np.array(loss_log)
        ax1 = plt.subplot(121)
        ax1.plot(loss_log)
        ax1.set_ylabel('Loss', weight = 'bold')
        acc_log = np.array(acc_log)
        ax2 = plt.subplot(111)
        ax2.plot(acc_log)
        ax2.set_ylabel('Train_accuracy', weight = 'bold')
        np.savetxt(X=np.vstack((loss_log, acc_log)), fname='loss_acc_log.txt', fmt='%.3f')